In [1]:
import os

In [2]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")

In [5]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.7,convert_system_message_to_human=True)

In [10]:
while True:
    question = input("Enter your question (or 'exit' to quit): ")
    if question.lower() not in ['exit', 'quit']:
        print(model.invoke(question).content)
    else:
        print("Exiting the program.")
        break

Exiting the program.


### Adding memory to the model

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage, HumanMessage

In [13]:
store = {}

In [14]:
def get_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [16]:
config = {
    "configurable": {"session_id": "first_session"}
}

In [17]:
model_with_memory = RunnableWithMessageHistory(model,get_session_id)

In [18]:
model_with_memory.invoke(("Hello I am John Wick."), config=config).content

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'Mr. Wick.\n\nA pleasure. I trust you\'re not "working" tonight.\n\nHow may I be of service? Are you in need of:\n\n*   A "dinner reservation"?\n*   A tactical map of the city?\n*   The finest veterinary services for your companion?\n*   Or simply a quiet room.\n\nMy services are at your disposal. Just say the word.'

In [19]:
model_with_memory.invoke(("Who am I?"), config=config).content

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'Mr. Wick.\n\nThat is a question with many answers, depending on who you ask.\n\nTo the criminal underworld, you are a legend. A myth. You are **Baba Yaga**. Not the Boogeyman, no... you are the one they send to *kill* the Boogeyman.\n\nTo men like Viggo Tarasov, you were the instrument of their ascension. The man who completed the "impossible task," allowing them to build an empire. You were their finest, most terrifying asset.\n\nTo the High Table, you are a disruption. A ghost who returned from the grave. A broken rule. A loose thread that threatens to unravel their entire tapestry. You are **Excommunicado**. A man with a multi-million dollar bounty on his head.\n\nTo those who have seen you work, you are a man of focus, commitment, and sheer, unadulterated will. The artist behind the story of three men in a bar, killed with a single pencil. You are a master of gun-fu, a tactician, a force of nature that cannot be stopped, only weathered.\n\nTo your late wife, Helen, you were simply

### Adding RAG functionality using LCEL

In [20]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

In [21]:
loader = DirectoryLoader('./data', glob="**/*.txt", loader_cls=TextLoader)

In [22]:
docs = loader.load()

In [24]:
docs

[Document(metadata={'source': 'data/info.txt'}, page_content='John Wick is an American media franchise created by Derek Kolstad. It centers on a neo-noir action thriller film series featuring the eponymous character portrayed by Keanu Reeves. Wick is a legendary hitman who is reluctantly drawn back into the criminal underworld after retiring. The franchise began with the release of John Wick (2014), which was followed by three sequels: Chapter 2 (2017), Chapter 3 – Parabellum (2019), and Chapter 4 (2023).[1][2] Various spin-offs expanded the franchise: the prequel comic book series John Wick: The Book of Rules (2017–2019), the prequel television miniseries The Continental (2023), and the spinoff film Ballerina (2025),[3] all incorporating elements of alternate history.\n\nThe films have received critical acclaim, and have been considered one of the greatest action film series of all time.[4][5][6][7] Some critics and publications consider the first film,[a] as well as Chapter 4,[25][26

In [25]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", " ", ""],
)

In [26]:
new_docs = text_splitter.split_documents(documents = docs)

In [27]:
doc_strings = [doc.page_content for doc in new_docs]

In [32]:
# Adding an embedding model to create vector store

from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [33]:
db = Chroma.from_documents(new_docs, embeddings)

In [34]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":4})

In [35]:
template = """
You are a helpful AI assistant. Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}

Question: {question}"""

prompt = PromptTemplate.from_template(template)

In [36]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

In [37]:
question = "What is John Wick's profession?"

In [39]:
print(retrieval_chain.invoke(question))

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Based on the provided context, John Wick is a legendary hitman.


## Tools and Agents

In [40]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

### Adding a Wikipedia Wrapper API tool

In [41]:
api_wrapper = WikipediaAPIWrapper()

In [42]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [45]:
tool.name

'wikipedia'

In [44]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [46]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [49]:
print(tool.run({"query": "Who is Iron Man?"}))

Page: Iron Man
Summary: Iron Man is a superhero appearing in American comic books published by Marvel Comics. Co-created by writer and editor Stan Lee, developed by scripter Larry Lieber, and designed by artists Don Heck and Jack Kirby, the character first appeared in Tales of Suspense #39 in 1962 (cover dated March 1963) and received his own title with Iron Man #1 in 1968. Shortly after his creation, Iron Man became a founding member of the superhero team, the Avengers, alongside Thor, Ant-Man, the Wasp, and the Hulk. Iron Man stories, individually and with the Avengers, have been published consistently since the character's creation.
Iron Man is the superhero persona of Anthony Edward "Tony" Stark, a businessman and engineer who runs the weapons manufacturing company Stark Industries. When Stark was captured in a war zone and sustained a severe heart wound, he built his Iron Man armor and escaped his captors. Iron Man's suits of armor grant him superhuman strength, flight, energy pro

### Adding a YouTube search tool

In [52]:
from langchain_community.tools import YouTubeSearchTool

In [53]:
tool1 = YouTubeSearchTool()

In [54]:
tool1.name

'youtube_search'

In [56]:
tool1.run({"query": "Python programming tutorials"})

"['https://www.youtube.com/watch?v=K5KVEU3aaeQ&pp=ygUcUHl0aG9uIHByb2dyYW1taW5nIHR1dG9yaWFscw%3D%3D', 'https://www.youtube.com/watch?v=kqtD5dpn9C8&pp=ygUcUHl0aG9uIHByb2dyYW1taW5nIHR1dG9yaWFscw%3D%3D']"

### Adding a Web Search Tool

In [58]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [59]:
tool2 = TavilySearchResults()

/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/ipykernel_21165/4121445016.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool2 = TavilySearchResults()


In [60]:
tool2.invoke({"query": "Was there a flood in Jammu Kashmir recently?"})

[{'title': '2025 Kishtwar district flash flood - Wikipedia',
  'url': 'https://en.wikipedia.org/wiki/2025_Kishtwar_district_flash_flood',
  'content': 'On 14 August 2025, a flash flood occurred in Chositi (also spelled Chashoti), a village in the Kishtwar district of India-administered Jammu and Kashmir "Jammu and Kashmir (union territory)"), as the result of a cloudburst. The incident caused at least 67 deaths, 300 injuries, and left around 200 people missing. In addition to residents, pilgrims on the Machail Mata Yatra pilgrimage route were also impacted.\n\n## Background [...] Wikipedia\nThe Free Encyclopedia\n\n## Contents\n\n# 2025 Kishtwar district flash flood\n\n2025 Kishtwar district flash flood\n\n|  |  |\n --- |\n| Date | 14 August 2025 (2025-08-14) |\n| Time | 11:30 (IST) |\n| Location | Chositi village, Kishtwar, Jammu and Kashmir "Jammu and Kashmir (union territory)"), India |\n| Type | Flash flood |\n| Cause | Cloudburst |\n| Deaths | 67+ |\n| Non-fatal injuries | 300+ |\

In [61]:
from langchain.agents import AgentType, initialize_agent, load_tools

In [65]:
agent = initialize_agent(
    tools=[tool, tool1, tool2],
    llm=model,                                   # ChatGoogleGenerativeAI(...)
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,                  # <-- auto-retry when parse fails
    max_iterations=3,
    early_stopping_method="generate"             # if stuck, it will produce "Final Answer:"
)

In [66]:
agent.run("What is the current population of the world?")



> Entering new AgentExecutor chain...


/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: I need to find the current population of the world. This is a value that changes constantly, so a real-time or very recent source is best. The `tavily_search_results_json` tool is the most suitable for finding up-to-date information and current events. Wikipedia might have a page on world population, but it might not be as current as a direct search.
Action: tavily_search_results_json
Action Input: "current world population"
Observation: [{'title': 'Current World Population: 8,005,176,000', 'url': 'https://worldpopulationreview.com/', 'content': 'today. [...] Articles about the ease, safety, and attractiveness of visiting a place for tourism or other purposes\n\n# Explore the World Population Through Data\n\nDiscover population, economy, health, and more with the most comprehensive global statistics\nat your fingertips.\n\n## Contents\n\n# Current World Population: 8,005,176,000\n\nIndia Flag\n\n1,463,870,000\n\nChina Flag\n\n1,416,100,000\n\nUnited States Flag\n\n347,276,000\

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


I now know the final answer
Final Answer: According to World Population Review, the current population of the world is approximately 8.005 billion people. Other sources, such as the U.S. Census Bureau and Statista, estimate the population to be between 8.1 and 8.2 billion in 2025.

> Finished chain.


'According to World Population Review, the current population of the world is approximately 8.005 billion people. Other sources, such as the U.S. Census Bureau and Statista, estimate the population to be between 8.1 and 8.2 billion in 2025.'

In [ ]:
from langchain_core.tools import tool

In [ ]:
@tool
def search(query: str):
    """Call to surf the web."""
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."

In [ ]:
tools = [search]

In [ ]:
tool_node = ToolNode(tools)

In [ ]:
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [ ]:
from typing import Literal

In [ ]:
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END

In [ ]:
workflow=StateGraph(MessagesState)

In [ ]:
workflow.add_node("agent",call_model)

In [ ]:
workflow.add_node("tools",call_model)

In [ ]:
# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

In [ ]:
workflow.set_entry_point("agent")

In [ ]:
# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')

In [ ]:
checkpointer = MemorySaver()

In [ ]:
app=workflow.compile(checkpointer=checkpointer)

In [ ]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
from langchain_core.messages import HumanMessage

In [ ]:
final_state = app.invoke({"messages": [HumanMessage(content="What is the weather like in Florida?")]},config={"configurable": {"thread_id": 42}})

In [ ]:
final_state["messages"][-1].content

In [ ]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    for event in app.stream({"messages": ("user", user_input)}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)